In [ ]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import plotly.express as px
from pathlib import Path
import holidays


In [ ]:
def upload_data():
    data_path = str(Path.cwd().parent) + "\\Data\\EPC\\Power Consumption Data.csv"

    df = pd.read_csv(data_path)

    df = df[df["real_consumption"] > 0]
    df = df[df['real_consumption'] <= df['real_consumption'].mean() + 4 * df['real_consumption'].std()]

    df['time'] = pd.to_datetime(df['time'])
    df = df.sort_values(by='time',ascending=True)

    return df

def data_metrics(data, real, predicted):

    y_true = data[real]
    y_pred = data[predicted]


    def r2_score_tf(y_true, y_pred):
        """
        Computes the R² (coefficient of determination) score in TensorFlow.
        
        Parameters:
        y_true : Tensor
            Ground truth values.
        y_pred : Tensor
            Predicted values.
    
        Returns:
        Tensor
            The R² score.
        """
        
        
        y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
    
        unexplained_error = tf.reduce_sum(tf.square(y_true - y_pred))  # SSE
        total_error = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))  # SST
    
        r_squared = 1.0 - (unexplained_error / (total_error + tf.keras.backend.epsilon()))  # Prevent division by zero
        return r_squared


    # Calculate metrics
    mae = tf.keras.losses.MeanAbsoluteError()(y_true, y_pred).numpy()
    mse = tf.keras.losses.MeanSquaredError()(y_true, y_pred).numpy()
    rmse = np.sqrt(mse)
    r2 = r2_score_tf(y_true, y_pred)


    # MAE (Mean Absolute Error):
    # Lower values are better; good MAE depends on the scale of 'real_consumption'.
    # As a rule of thumb, MAE should be significantly smaller than the mean of the target variable.
    # Lower is better. Ideally, MAE should be much less than the average value of y_true.
    print(f"MAE: {mae:.4f}")

    # MSE (Mean Squared Error):
    # Similar to MAE but penalizes large errors more heavily. A smaller MSE is better.
    # Compare MSE to the variance of 'real_consumption' for context.
    # Lower is better. MSE should ideally be close to zero relative to the variance of y_true.
    print(f"MSE: {mse:.4f}")



    # RMSE (Root Mean Squared Error):
    # RMSE is the square root of MSE and is in the same units as 'real_consumption'.
    # A good RMSE is often close to the standard deviation of 'real_consumption'.
    # Lower is better. RMSE should be comparable to or less than the standard deviation of y_true."
    print(f"RMSE: {rmse:.4f}")



    # R² (Coefficient of Determination):
    # R² measures how well the predictions explain the variability of the data.
    # Values close to 1.0 are excellent, indicating the model explains most of the variance.
    # Negative values indicate poor fit.
    # Closer to 1.0 is better. Values > 0.7 are generally good; < 0.5 indicates underfitting.
    print(f"R²: {r2:.4f}")

def feature_engineering(data):

    # Extracting basic time-based features
    data['hour'] = data['time'].dt.hour  # Hour of the day
    data['minute'] = data['time'].dt.minute  # Minute
    data['day_of_week'] = data['time'].dt.dayofweek  # Day of the week (0=Monday, 6=Sunday)
    data['is_weekend'] = data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)  # Weekend flag
    data['day_of_month'] = data['time'].dt.day
    data['week_of_year'] = data['time'].dt.isocalendar().week
    data['month'] = data['time'].dt.month
    data['quarter'] = data['time'].dt.quarter
    data['year'] = data['time'].dt.year


    # Generate lag features for temporal dependency modeling
    for lag in range(1, 5):  # Create lag features for the past 4 time steps
        data[f'lag_{lag}'] = data['real_consumption'].shift(lag)


    # Generate exponential moving averages
    for span in [3, 5]:  # Spans of size 3, 5, and 7
        data[f'ema_{span}'] = data['real_consumption'].ewm(span=span, adjust=False).mean()

    # Rolling average over a longer period (e.g., weekly and monthly moving averages)
    data['weekly_avg'] = data['real_consumption'].rolling(window=7*24*20, min_periods=1).mean()  # Weekly moving avg
    data['monthly_avg'] = data['real_consumption'].rolling(window=30*24*20, min_periods=1).mean()  # Monthly moving avg


    # Percentage change in real consumption
    data['pct_change'] = data['real_consumption'].pct_change()

    data['hour_sin'] = np.sin(2 * np.pi * data['hour'] / 24)  # Cyclic hour feature (sine)
    data['hour_cos'] = np.cos(2 * np.pi * data['hour'] / 24)  # Cyclic hour feature (cosine)

    data['day_of_week_sin'] = np.sin(2 * np.pi * data['day_of_week'] / 7)
    data['day_of_week_cos'] = np.cos(2 * np.pi * data['day_of_week'] / 7)

    data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
    data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)

    data['week_of_year_sin'] = np.sin(2 * np.pi * data['week_of_year'] / 52)
    data['week_of_year_cos'] = np.cos(2 * np.pi * data['week_of_year'] / 52)




    # Get Georgia holidays for all years in the dataset
    georgia_holidays = holidays.Georgia(years=range(data["year"].min(), data["year"].max() + 1))

    data["date"] = data["time"].dt.date

    # Create holiday feature (1 if it's a holiday, 0 otherwise)
    data['is_holiday'] = data["date"].map(lambda x: 1 if x in georgia_holidays else 0)

    # Add features for the day before and after a holiday
    data['is_day_before_holiday'] = data["date"].map(lambda x: 1 if (x - pd.Timedelta(days=1)) in georgia_holidays else 0)
    data['is_day_after_holiday'] = data["date"].map(lambda x: 1 if (x + pd.Timedelta(days=1)) in georgia_holidays else 0)



    return  data



In [ ]:
df = upload_data()

data_metrics(data=df, real="real_consumption", predicted="predicted_consumption")

df = feature_engineering(df)


In [ ]:
df.index = pd.to_datetime(df['time'], format='%d.%m.%Y %H:%M:%S')
temp = df['real_consumption']
temp

In [ ]:
# [[[1], [2], [3], [4], [5]]] [6]
# [[[2], [3], [4], [5], [6]]] [7]
# [[[3], [4], [5], [6], [7]]] [8]

def df_to_X_y(df, window_size=5):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [[a] for a in df_as_np[i:i+window_size]]
        X.append(row)
        label = df_as_np[i+window_size]
        y.append(label)
    return np.array(X), np.array(y)

In [ ]:
WINDOW_SIZE = 5
X1, y1 = df_to_X_y(temp, WINDOW_SIZE)
X1.shape, y1.shape

In [ ]:
X_train1, y_train1 = X1[:600000], y1[:600000]
X_val1, y_val1 = X1[600000:650000], y1[600000:650000]
X_test1, y_test1 = X1[650000:], y1[650000:]
X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape

In [ ]:
model1 = Sequential()
model1.add(InputLayer((WINDOW_SIZE, 1)))
model1.add(LSTM(64))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'linear'))

model1.summary()

In [ ]:
from pathlib import Path
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError

# Define model checkpoint path with a valid .keras filename
checkpoint_path = str(Path.cwd().parent / "Data" / "best_model.keras")

cp1 = ModelCheckpoint(
    filepath=checkpoint_path,  # Ensure it ends with .keras
    save_best_only=True
)

model1.compile(
    loss=MeanSquaredError(),
    optimizer=Adam(learning_rate=0.0001),
    metrics=[RootMeanSquaredError()]
)


In [10]:
model1.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=20, callbacks=[cp1])

18750/18750 ━━━━━━━━━━━━━━━━━━━━ 109s 6ms/step - loss: 322.2142 - root_mean_squared_error: 17.9500 - val_loss: 327.5891 - val_root_mean_squared_error: 18.0994
Epoch 19/20
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 111s 6ms/step - loss: 331.1414 - root_mean_squared_error: 18.1855 - val_loss: 340.5874 - val_root_mean_squared_error: 18.4550
Epoch 20/20
18750/18750 ━━━━━━━━━━━━━━━━━━━━ 112s 6ms/step - loss: 321.9810 - root_mean_squared_error: 17.9437 - val_loss: 366.0849 - val_root_mean_squared_error: 19.1333


In [ ]:
from tensorflow.keras.models import load_model
model1 = load_model(checkpoint_path)

train_predictions = model1.predict(X_train1).flatten()
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y_train1})

val_predictions = model1.predict(X_val1).flatten()
val_results = pd.DataFrame(data={'Val Predictions':val_predictions, 'Actuals':y_val1})

test_predictions = model1.predict(X_test1).flatten()
test_results = pd.DataFrame(data={'Test Predictions':test_predictions, 'Actuals':y_test1})


18750/18750 ━━━━━━━━━━━━━━━━━━━━ 48s 3ms/step
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step
1475/1823 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step

In [ ]:
test_results["index"] = temp[650005:].index
test_results

In [ ]:
predict_points = 100
test_predictions = []
last_several_month = temp[650005:650011].copy()


current_batch = df_to_X_y(last_several_month, window_size=5)[0]

for i in range(predict_points):

    # get the prediction value for the first batch
    current_pred = model1.predict(current_batch).flatten()

    # append the prediction into the array
    test_predictions.append(current_pred)

    # use the prediction to update the batch and remove the first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)



results = temp[650010:(650010 + predict_points)].copy().reset_index()
results["predic"] = pd.DataFrame(test_predictions).rename(columns={0:"predict"})["predict"]
results

In [ ]:
# Create an interactive plot using Plotly
fig = px.line(results, x='time', y=['real_consumption', 'predic'], title="Interactive Time Series Plot")

# Customize the plot
fig.update_layout(
    xaxis_title="Timestamp",
    yaxis_title="Values",
    legend_title="Legend",
    xaxis_rangeslider_visible=True  # Enables the date slicer
)

# Show the plot
fig.show()

In [ ]:
data_metrics(data=test_results, real="Actuals", predicted="Test Predictions")